In [9]:
import numpy as numpy
from matplotlib import pyplot as matplot
%matplotlib inline
import pandas as pandas
import math
from sklearn import preprocessing
from sklearn import svm
import cv2
import os
import ntpath
from sklearn.metrics.pairwise import cosine_similarity
import fnmatch
import glob
import math
from math import sqrt,log
from cmath import sqrt
from keras import layers
from keras.layers import Input, Add, Multiply, Subtract, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import Dense, Dropout, Input, Lambda, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import l2
from keras.models import Model, load_model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
# import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform, he_normal
import scipy.misc
import keras.backend as K
K.set_image_data_format('channels_last') 
K.set_learning_phase(1) 
import scipy
import cv2
from keras.layers import subtract
import keras
from keras.layers import Lambda
import tensorflow as tf 
from keras.optimizers import SGD, RMSprop, Adadelta

In [4]:
def preprocessing_input(X_input):
    scipy.misc.imresize(X_input, (155,220), interp='bilinear')
    X_input=cv2.bitwise_not(X_input)
    X_input=X_input/245
    return X_input
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def euclidean_distance_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)
def contrastive_loss(y_true,y_pred):
    margin=1
    L=K.mean((1-y_true) * K.square(y_pred) + (y_true) * K.square(K.maximum(1-y_pred, 0)))
    return L
def sigmodel(input_shape):
    
    seq = Sequential()
    seq.add(Convolution2D(96, 11, 11, activation='relu', name='conv11', subsample=(4, 4), input_shape= input_shape, 
                        init='he_normal'))
    seq.add(BatchNormalization(epsilon=1e-06, mode=0, axis=1, momentum=0.9))
    seq.add(MaxPooling2D((3,3), strides=(2, 2)))    
    seq.add(ZeroPadding2D((2, 2)))
    
    seq.add(Convolution2D(256, 5, 5, activation='relu', name='conv21', subsample=(1, 1), init='he_normal'))
    seq.add(BatchNormalization(epsilon=1e-06, mode=0, axis=1, momentum=0.9))
    seq.add(MaxPooling2D((3,3), strides=(2, 2)))
    seq.add(Dropout(0.3))
    seq.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    
    seq.add(Convolution2D(384, 3, 3, activation='relu', name='conv31', subsample=(1, 1), init='he_normal'))
    seq.add(ZeroPadding2D((1, 1), dim_ordering='tf'))
    
    seq.add(Convolution2D(256, 3, 3, activation='relu', name='conv32', subsample=(1, 1), init='he_normal'))    
    seq.add(MaxPooling2D((3,3), strides=(2, 2)))
    seq.add(Dropout(0.3))
    seq.add(Flatten(name='flatten'))
    seq.add(Dense(1024, W_regularizer=l2(0.0005), activation='relu', init=glorot_uniform(seed=0)))
    seq.add(Dropout(0.3))
    
    seq.add(Dense(128, W_regularizer=l2(0.0005), activation='relu', init=glorot_uniform(seed=0))) 
    
    return seq

In [10]:
Genuinedir = 'C:\Siddhartha\Confidential Documents\Stretch Projects\Data Science\Signature Verification\Train_Genuine'
os.chdir(Genuinedir)
Original_Signature_Table = pandas.read_csv("Training.csv")

In [15]:
X_train = Original_Signature_Table.iloc[0:,2:27]

In [17]:
X_train = numpy.asarray(X_train)

In [36]:
Y_train=np.array([])
for m in range(13800):
    Y_train=np.insert(Y_train,0,1)

In [37]:
Y_train

array([1., 1., 1., ..., 1., 1., 1.])

In [38]:
apply_sigmodel = sigmodel((155,220,3))
    
X_input1 = Input(shape=(155,220,3))
X_input2 = Input(shape=(155,220,3)) 

X_vect1 = apply_sigmodel(X_input1)
X_vect2 = apply_sigmodel(X_input2)
    
distance = Lambda(euclidean_distance, output_shape=euclidean_distance_output_shape)([X_vect1,X_vect2])
    
model = Model(input=[X_input1,X_input2], output=distance)

rms = RMSprop(lr=1e-4, rho=0.9, epsilon=1e-08)

model.compile(optimizer=rms,loss=contrastive_loss,metrics=['accuracy'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (11, 11), activation="relu", name="conv11", input_shape=(155, 220,..., strides=(4, 4), kernel_initializer="he_normal")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 5), activation="relu", name="conv21", strides=(1, 1), kernel_initializer="he_normal")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `ZeroPadding2D` ca

In [39]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 10)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[ 3.12387706e-02,  2.00000000e+00,  7.32276619e-01, ...,
        -3.13800000e-01,  7.10100000e+03,  2.40000000e+01],
       [ 3.41956431e-02,  3.00000000e+00,  7.39615987e-01, ...,
        -5....

In [40]:
import h5py
model.save_weights('model_weights33.h5') 

In [41]:
model.load_weights('my_model3.h5', by_name=False) 

OSError: Unable to open file (Unable to open file: name = 'my_model3.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)